1. Inspection du schema
2. Echantillonnage?
3. Création des 3 datasets
4. Exploration rapide pour connaitre les données
5. Création du fichier client (donc des features)
6. Exploration des différents types de profils de client
7. Exploration du potentiel prédictif des features
8. Split
9. Feature Eng

## <a name="C4">0. Importation et fonctions</a>

### 0.1 Importation des librairies

In [1]:
import warnings
warnings.filterwarnings('ignore')

import sqlite3
import pandas as pd

### 0.2 Importation des données

In [2]:
olist_db = sqlite3.connect('data/olist.db')

In [3]:
cursor = olist_db.cursor()

## <a name="C4">1. Préparation des données</a>

### 1.1 Inspection du schéma

J'inspecte d'abord l'organisation de la base de données avant de rentrer dans l'exploration des tables pertinentes.  
D'abord, je liste les tables.

In [4]:
tables_list = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables_list = [table[0] for table in tables_list.fetchall()]
tables_list

['customers',
 'geoloc',
 'order_items',
 'order_pymts',
 'order_reviews',
 'orders',
 'products',
 'sellers',
 'translation']

Pour avoir un ordre d'idée de l'ampleur du jeu de données, j'affiche le nombre de lignes par table.

In [5]:
tables_rows_count = []
for table in tables_list:
    n = cursor.execute(f"SELECT COUNT(*) FROM {table};").fetchall()[0][0]
    tables_rows_count.append(n)
    print(f"{table}: {n} rows")

customers: 99441 rows
geoloc: 1000163 rows
order_items: 112650 rows
order_pymts: 103886 rows
order_reviews: 99224 rows
orders: 99441 rows
products: 32951 rows
sellers: 3095 rows
translation: 71 rows


Je peux maintenant inspecter le schéma de chaque table.

In [6]:
tables_schema = [cursor.execute(f"PRAGMA table_info({table})").fetchall() for table in tables_list]
tables = [(tables_list[i], tables_rows_count[i], tables_schema[i]) for i in range(len(tables_list))]
tables

[('customers',
  99441,
  [(0, 'index', 'BIGINT', 0, None, 0),
   (1, 'customer_id', 'TEXT', 0, None, 0),
   (2, 'customer_unique_id', 'TEXT', 0, None, 0),
   (3, 'customer_zip_code_prefix', 'BIGINT', 0, None, 0),
   (4, 'customer_city', 'TEXT', 0, None, 0),
   (5, 'customer_state', 'TEXT', 0, None, 0)]),
 ('geoloc',
  1000163,
  [(0, 'index', 'BIGINT', 0, None, 0),
   (1, 'geolocation_zip_code_prefix', 'BIGINT', 0, None, 0),
   (2, 'geolocation_lat', 'FLOAT', 0, None, 0),
   (3, 'geolocation_lng', 'FLOAT', 0, None, 0),
   (4, 'geolocation_city', 'TEXT', 0, None, 0),
   (5, 'geolocation_state', 'TEXT', 0, None, 0)]),
 ('order_items',
  112650,
  [(0, 'index', 'BIGINT', 0, None, 0),
   (1, 'order_id', 'TEXT', 0, None, 0),
   (2, 'order_item_id', 'BIGINT', 0, None, 0),
   (3, 'product_id', 'TEXT', 0, None, 0),
   (4, 'seller_id', 'TEXT', 0, None, 0),
   (5, 'shipping_limit_date', 'TEXT', 0, None, 0),
   (6, 'price', 'FLOAT', 0, None, 0),
   (7, 'freight_value', 'FLOAT', 0, None, 0)]),
 (

Via ce schéma et la description des colonnes disponibles sur Kaggle, je distingue plusieurs informations qui pourraient être pertinentes lors d'une segmentation client:  
- Informations géographiques (*customer*, *geoloc*)
- Montants des achats (*order_items*, *order_pymts*)
- Catégorie de produits achetés (*order_items*, *products*)
- Périodicité et récence des achats (*orders*)
- Méthode et type de paiement (*order_pymts*)
- Satisfaction (*order_reviews*)

### 1.2 Création des datasets et première sélection des features

Pour faciliter l'exploration puis le travail des features, je vais créer plusieurs tables qui représentent et joignent les informations citées juste au-dessus, un par niveau de granularité présent dans la base de données.  
Ici, j'en distingue 3:  
- Les clients, contenant leurs coordonnées géographiques
- Les commandes, contenant leur informations de tracking, de paiement, de montant, de satisfaction
- Les produits commandés, contenant leurs informations liés à la commande associée (quantité, prix), et leurs caractéristiques

Je vais donc garder uniquement les colonnes représentant une de ces informations.

L'objectif lors du Feature Engineering sera de constituer un unique fichier client, en réalisant des aggrégations de ces informations pour créer des features qui seront toutes au même niveau de granularité.  
Ici, je ne fais pour l'instant que des jointures, pour permettre d'explorer plus facilement.

In [13]:

command = """
SELECT 
customer_id, 
customer_unique_id, 
customer_zip_code_prefix AS zip_code, 
customer_city AS city,
customer_state AS state
FROM customers
"""

customer_df = pd.read_sql_query(command, olist_db)
customer_df

,customer_id,customer_unique_id,zip_code,city,state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [16]:
command = """
SELECT
order_id,
a.customer_id,
a.order_status,
a.order_purchase_timestamp,
a.order_approved_at,
b.payment_sequential,
b.payment_type,
b.payment_installments,
b.payment_value,
c.review_score
FROM orders AS a
LEFT JOIN order_pymts AS b USING(order_id)
LEFT JOIN order_reviews AS c USING(order_id)
"""

orders_df = pd.read_sql_query(command, olist_db)
orders_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,payment_sequential,payment_type,payment_installments,payment_value,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,1.0,credit_card,1.0,18.12,4.0
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2.0,voucher,1.0,18.59,4.0
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,3.0,voucher,1.0,2.00,4.0
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,1.0,boleto,1.0,141.46,4.0
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,1.0,credit_card,3.0,179.12,5.0
...,...,...,...,...,...,...,...,...,...,...
104473,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,1.0,credit_card,3.0,85.08,5.0
104474,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,1.0,credit_card,3.0,195.00,4.0
104475,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,1.0,credit_card,5.0,271.01,5.0
104476,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,1.0,credit_card,4.0,441.16,2.0


In [18]:
command = """
SELECT
product_id,
a.order_id,
a.order_item_id AS quantity,
a.price,
a.freight_value,
b.product_category_name
FROM order_items AS a
LEFT JOIN products AS b USING(product_id)
"""

products_df = pd.read_sql_query(command, olist_db)
products_df

,product_id,order_id,quantity,price,freight_value,product_category_name
0,4244733e06e7ecb4970a6e2683c13e61,00010242fe8c5a6d1ba2dd792cb16214,1,58.90,13.29,cool_stuff
1,e5f2d52b802189ee658865ca93d83a8f,00018f77f2f0320c557190d7a144bdd3,1,239.90,19.93,pet_shop
2,c777355d18b72b67abbeef9df44fd0fd,000229ec398224ef6ca0657da4fc703e,1,199.00,17.87,moveis_decoracao
3,7634da152a4610f1595efa32f14722fc,00024acbcdf0a6daa1e931b038114c75,1,12.99,12.79,perfumaria
4,ac6c3623068f30de03045865e4e10089,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90,18.14,ferramentas_jardim
...,...,...,...,...,...,...
112645,4aa6014eceb682077f9dc4bffebc05b0,fffc94f6ce00a00581880bf54a75a037,1,299.99,43.41,utilidades_domesticas
112646,32e07fd915822b0765e448c4dd74c828,fffcd46ef2263f404302a634eb57f7eb,1,350.00,36.53,informatica_acessorios
112647,72a30483855e2eafc67aee5dc2560482,fffce4705a9662cd70adb13d4a31832d,1,99.90,16.95,esporte_lazer
112648,9c422a519119dcad7575db5af1ba540e,fffe18544ffabc95dfada21779c9644f,1,55.99,8.72,informatica_acessorios


### 1.3 Première exploration

- Par commande
- Par produit

## <a name="C4">2. Création du fichier client</a>

### 2.1 Création des features

### 2.2 Exploration des différents profils de clients

### 2.3 Analyse du potentiel prédictif

## 3. Préparation au clustering

### 3.1 Split Train/Test

### 3.2 Transformation des features